<h1><center>Big Data Algorithms Techniques & Platforms</center></h1>
<h2>
<hr style=" border:none; height:3px;">
<center>Spark and DataFrames</center>
<hr style=" border:none; height:3px;">
</h2>

## Objectives 

<strong> Dataframes: </strong>
<ul>
    <li>  PySpark </li> 
    <li>  Pandas library on Spark</li> 
</ul>

# A. Context

<p align="justify">
<font size="3">
We are going to use quite a big dataset containing data on Bitcoin available on <a href="https://www.kaggle.com/mczielinski/bitcoin-historical-data">Kaggle</a>.

As stated in the description of the dataset:
"Bitcoin is the longest running and most well-known cryptocurrency, first released as open source in 2009 by the anonymous Satoshi Nakamoto. Bitcoin serves as a decentralized medium of digital exchange, with transactions verified and recorded in a public distributed ledger (the blockchain) without the need for a trusted record keeping authority or central intermediary." 
</font>
</p>


### The dataset

<p align="justify">
<font size="3">
The dataset is in a .csv file for selected bitcoin exchanges for the time period of January 2012 to March 2021, with minute-to-minute updates of OHLC (Open, High, Low, Close), Volume in BTC and indicated currency, and weighted bitcoin price. 

Notice that:
<ul>
    <li> Timestamps are in Unix time.</li>
<li> Timestamps without any trades or activity have their data fields filled with NaNs. </li>
<li>  If a timestamp is missing, or if there are jumps, this may be because the exchange (or its API) was down, the exchange (or its API) did not exist, or some other unforeseen technical error in data reporting or gathering. </li>
</ul>
As stated by the author, "all effort has been made to deduplicate entries and verify the contents are correct and complete to the best of my ability, but obviously trust at your own risk".
</p>
</font>

# B. Environment set-up

In [1]:
# !apt-get update
# !apt-get install openjdk-8-jdk-headless -qq > /dev/null
# !wget -q https://downloads.apache.org/spark/spark-3.0.3/spark-3.0.3-bin-hadoop2.7.tgz
# !tar zxvf spark-3.0.3-bin-hadoop2.7.tgz
# !pip install -q findspark
# !pip install plotly
# !unzip archive.zip

import os
import findspark
import pandas as pd
from pandas.testing import assert_frame_equal
from plotly.offline import iplot, init_notebook_mode
import plotly.graph_objects as go
import datetime as dt

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.3-bin-hadoop2.7"
findspark.init()

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import TimestampType
from pyspark.sql.functions import (
    to_timestamp, year, month, min as spark_min, max as spark_max, mean, stddev)

spark = SparkSession.builder\
    .appName("Assignment2")\
    .getOrCreate()

## B.1  File import
    
<p align="justify">
<font size="3">
In this exercise, the goal is to create a Spark DataFrame from the csv file in input. 

Recall that in a Spark DataFrame, the type of a column is very important for the internal data representation. 
    
For this step, use the following types for the columns:
<ul>
    <li>    $Date_Time: Timestamp$ </li>
     <li>   $Open: double$ </li>
     <li>   $High: double$ </li>
    <li>    $Low: double$ </li>
    <li>    $Close: double$ </li>
    <li>    $Volume_BTC: double$ </li>
    <li>    $Volume_Currency: double$ </li>
    <li>    $Weighted_Price: double$ </li>
</ul>
    
</font>
</p>

### <strong> Exercise 1.</strong> First import (1 point)
    
<p align="justify">
<font size="3">
Import the csv file into a Spark DataFrame.
</font>
</p>

In [3]:
df1 = spark.read.option("header", "true")\
    .option("inferSchema", "true")\
    .csv("bitstampUSD_1-min_data_2012-01-01_to_2021-03-31.csv")
df1

#######################
# EXPECTED OUTPUT:
# DataFrame[Timestamp: int, Open: double, High: double, Low: double, Close: double, Volume_(BTC): double, Volume_(Currency): double, Weighted_Price: double]
#######################

DataFrame[Timestamp: int, Open: double, High: double, Low: double, Close: double, Volume_(BTC): double, Volume_(Currency): double, Weighted_Price: double]

In [4]:
df1.take(5)

[Row(Timestamp=1325317920, Open=4.39, High=4.39, Low=4.39, Close=4.39, Volume_(BTC)=0.45558087, Volume_(Currency)=2.0000000193, Weighted_Price=4.39),
 Row(Timestamp=1325317980, Open=nan, High=nan, Low=nan, Close=nan, Volume_(BTC)=nan, Volume_(Currency)=nan, Weighted_Price=nan),
 Row(Timestamp=1325318040, Open=nan, High=nan, Low=nan, Close=nan, Volume_(BTC)=nan, Volume_(Currency)=nan, Weighted_Price=nan),
 Row(Timestamp=1325318100, Open=nan, High=nan, Low=nan, Close=nan, Volume_(BTC)=nan, Volume_(Currency)=nan, Weighted_Price=nan),
 Row(Timestamp=1325318160, Open=nan, High=nan, Low=nan, Close=nan, Volume_(BTC)=nan, Volume_(Currency)=nan, Weighted_Price=nan)]

### <strong> Exercise 2. </strong> Timestamp column (1 point)
    
<p align="justify">
<font size="3">
Look again at the target schema:
    
<ul>
    <li>    $Date\_Time: Timestamp$ </li>
     <li>   $Open: double$ </li>
     <li>   $High: double$ </li>
    <li>    $Low: double$ </li>
    <li>    $Close: double$ </li>
    <li>    $Volume\_BTC: double$ </li>
    <li>    $Volume\_Currency: double$ </li>
    <li>    $Weighted\_Price: double$ </li>
</ul>
    
You notice that the import data has three problems with respect to the target schema:
    
    
<ul>
    <li> the $Date\_Time$ column is not present in the original file </li>
    <li> there is an $int$ column $Timestamp$ that can be converted and transformed to a $Date$</li> 
    <li> some of the column names contain not required parentesis. </li>
</ul>     
</font>
</p>   
  
<p align="justify">
<font size="3">
Refine the import of the csv file and convert the "timestamp" column in the proper $Timestamp$ type:
    <ul>
        <li>   Create a new column <code>Date\_Time</code>, which is the conversion of the $Integer$ column into $Timestamp$ type  </li>
</ul>
</font>
</p>

In [5]:
df2 = df1.withColumn("Date_Time", to_timestamp("Timestamp")) 
df2

#######################
# EXPECTED OUTPUT:
# DataFrame[Timestamp: int, Open: double, High: double, Low: double, Close: double, Volume_(BTC): double, Volume_(Currency): double, Weighted_Price: double, Date_Time: timestamp]
#######################

DataFrame[Timestamp: int, Open: double, High: double, Low: double, Close: double, Volume_(BTC): double, Volume_(Currency): double, Weighted_Price: double, Date_Time: timestamp]

In [6]:
df2.take(5)

[Row(Timestamp=1325317920, Open=4.39, High=4.39, Low=4.39, Close=4.39, Volume_(BTC)=0.45558087, Volume_(Currency)=2.0000000193, Weighted_Price=4.39, Date_Time=datetime.datetime(2011, 12, 31, 7, 52)),
 Row(Timestamp=1325317980, Open=nan, High=nan, Low=nan, Close=nan, Volume_(BTC)=nan, Volume_(Currency)=nan, Weighted_Price=nan, Date_Time=datetime.datetime(2011, 12, 31, 7, 53)),
 Row(Timestamp=1325318040, Open=nan, High=nan, Low=nan, Close=nan, Volume_(BTC)=nan, Volume_(Currency)=nan, Weighted_Price=nan, Date_Time=datetime.datetime(2011, 12, 31, 7, 54)),
 Row(Timestamp=1325318100, Open=nan, High=nan, Low=nan, Close=nan, Volume_(BTC)=nan, Volume_(Currency)=nan, Weighted_Price=nan, Date_Time=datetime.datetime(2011, 12, 31, 7, 55)),
 Row(Timestamp=1325318160, Open=nan, High=nan, Low=nan, Close=nan, Volume_(BTC)=nan, Volume_(Currency)=nan, Weighted_Price=nan, Date_Time=datetime.datetime(2011, 12, 31, 7, 56))]

### <strong> Exercise 3.</strong> Column names (2 points)


<p align="justify">
<font size="3">
As you can see from the output of the previous exercise, the names of the columns still present some problems since there are some unnecessary parentheses.
    <ul>
     <li> Remove the not required parentheses from the colum names </li>
</ul>
</font>
</p>


In [7]:
df_no_paren = df2.withColumnRenamed("Volume_(BTC)", "Volume_BTC")\
              .withColumnRenamed("Volume_(Currency)", "Volume_Currency")
df_no_paren

#######################
# EXPECTED OUTPUT:
#DataFrame[Timestamp: int, Open: double, High: double, Low: double, Close: double Volume_BTC: double, Volume_Currency: double, Weighted_Price: double, Date_Time: timestamp]
#######################

DataFrame[Timestamp: int, Open: double, High: double, Low: double, Close: double, Volume_BTC: double, Volume_Currency: double, Weighted_Price: double, Date_Time: timestamp]

In [8]:
df_no_paren.show(5)

#######################
# Expected output:
#+----------+----+----+----+-----+----------+---------------+--------------+-------------------+
#| Timestamp|Open|High| Low|Close|Volume_BTC|Volume_Currency|Weighted_Price|          Date_Time|
#+----------+----+----+----+-----+----------+---------------+--------------+-------------------+
#|1325317920|4.39|4.39|4.39| 4.39|0.45558087|   2.0000000193|          4.39|2011-12-31 07:52:00|
#|1325317980| NaN| NaN| NaN|  NaN|       NaN|            NaN|           NaN|2011-12-31 07:53:00|
#|1325318040| NaN| NaN| NaN|  NaN|       NaN|            NaN|           NaN|2011-12-31 07:54:00|
#|1325318100| NaN| NaN| NaN|  NaN|       NaN|            NaN|           NaN|2011-12-31 07:55:00|
#|1325318160| NaN| NaN| NaN|  NaN|       NaN|            NaN|           NaN|2011-12-31 07:56:00|
#+----------+----+----+----+-----+----------+---------------+--------------+-------------------+

+----------+----+----+----+-----+----------+---------------+--------------+-------------------+
| Timestamp|Open|High| Low|Close|Volume_BTC|Volume_Currency|Weighted_Price|          Date_Time|
+----------+----+----+----+-----+----------+---------------+--------------+-------------------+
|1325317920|4.39|4.39|4.39| 4.39|0.45558087|   2.0000000193|          4.39|2011-12-31 07:52:00|
|1325317980| NaN| NaN| NaN|  NaN|       NaN|            NaN|           NaN|2011-12-31 07:53:00|
|1325318040| NaN| NaN| NaN|  NaN|       NaN|            NaN|           NaN|2011-12-31 07:54:00|
|1325318100| NaN| NaN| NaN|  NaN|       NaN|            NaN|           NaN|2011-12-31 07:55:00|
|1325318160| NaN| NaN| NaN|  NaN|       NaN|            NaN|           NaN|2011-12-31 07:56:00|
+----------+----+----+----+-----+----------+---------------+--------------+-------------------+
only showing top 5 rows



## B.2 DataFrame columns 


<p align="justify">
<font size="3">
    
In this part of the exercise, we are going continue to modify the Spark DataFrames. Remember that using  PySpark, it's possible to access a DataFrame's columns either by attribute (<code>df.attributeName</code>) or by indexing <code>(df['attributeName'])</code>.
</font>
</p>


### <strong> Exercise 4.</strong>  Add two new columns to the DataFrame (2 points)
    
<p align="justify">
<font size="3">
We want to extend the DataFrame with two other columns: given the $Date\_Time$ column, you should create two new columns ($Year$ and $Month$) that contain 
    <ul>
     <li> the year </li>
     <li> the month of the year </li>
</ul>
    
</font>
</p>
    
    
    

In [9]:
df_with_date = df_no_paren.withColumn("Year", year("Date_Time"))\
                          .withColumn("Month", month("Date_Time"))

df_with_date.show(5)

#######################
# Expected output:
#+----------+----+----+----+-----+----------+---------------+--------------+-------------------+----+-----+
#| Timestamp|Open|High| Low|Close|Volume_BTC|Volume_Currency|Weighted_Price|          Date_Time|Year|Month|
#+----------+----+----+----+-----+----------+---------------+--------------+-------------------+----+-----+
#|1325317920|4.39|4.39|4.39| 4.39|0.45558087|   2.0000000193|          4.39|2011-12-31 07:52:00|2011|   12|
#|1325317980| NaN| NaN| NaN|  NaN|       NaN|            NaN|           NaN|2011-12-31 07:53:00|2011|   12|
#|1325318040| NaN| NaN| NaN|  NaN|       NaN|            NaN|           NaN|2011-12-31 07:54:00|2011|   12|
#|1325318100| NaN| NaN| NaN|  NaN|       NaN|            NaN|           NaN|2011-12-31 07:55:00|2011|   12|
#|1325318160| NaN| NaN| NaN|  NaN|       NaN|            NaN|           NaN|2011-12-31 07:56:00|2011|   12|
#+----------+----+----+----+-----+----------+---------------+--------------+-------------------+----+-----+


+----------+----+----+----+-----+----------+---------------+--------------+-------------------+----+-----+
| Timestamp|Open|High| Low|Close|Volume_BTC|Volume_Currency|Weighted_Price|          Date_Time|Year|Month|
+----------+----+----+----+-----+----------+---------------+--------------+-------------------+----+-----+
|1325317920|4.39|4.39|4.39| 4.39|0.45558087|   2.0000000193|          4.39|2011-12-31 07:52:00|2011|   12|
|1325317980| NaN| NaN| NaN|  NaN|       NaN|            NaN|           NaN|2011-12-31 07:53:00|2011|   12|
|1325318040| NaN| NaN| NaN|  NaN|       NaN|            NaN|           NaN|2011-12-31 07:54:00|2011|   12|
|1325318100| NaN| NaN| NaN|  NaN|       NaN|            NaN|           NaN|2011-12-31 07:55:00|2011|   12|
|1325318160| NaN| NaN| NaN|  NaN|       NaN|            NaN|           NaN|2011-12-31 07:56:00|2011|   12|
+----------+----+----+----+-----+----------+---------------+--------------+-------------------+----+-----+
only showing top 5 rows



###  <strong>Exercise 5.</strong>  Drop Timestamp (2 points)
    
<p align="justify">
<font size="3">
Finally we clean the schema and we can remove the the $Timestamp$ column.
</font>
</p>


In [10]:
df_clean = df_with_date.drop("Timestamp")
df_clean.show(5)

#######################
# Expected output:
#+----+----+----+-----+----------+---------------+--------------+-------------------+----+-----+
#|Open|High| Low|Close|Volume_BTC|Volume_Currency|Weighted_Price|          Date_Time|Year|Month|
#+----+----+----+-----+----------+---------------+--------------+-------------------+----+-----+
#|4.39|4.39|4.39| 4.39|0.45558087|   2.0000000193|          4.39|2011-12-31 07:52:00|2011|   12|
#| NaN| NaN| NaN|  NaN|       NaN|            NaN|           NaN|2011-12-31 07:53:00|2011|   12|
#| NaN| NaN| NaN|  NaN|       NaN|            NaN|           NaN|2011-12-31 07:54:00|2011|   12|
#| NaN| NaN| NaN|  NaN|       NaN|            NaN|           NaN|2011-12-31 07:55:00|2011|   12|
#| NaN| NaN| NaN|  NaN|       NaN|            NaN|           NaN|2011-12-31 07:56:00|2011|   12|
#+----+----+----+-----+----------+---------------+--------------+-------------------+----+-----+


+----+----+----+-----+----------+---------------+--------------+-------------------+----+-----+
|Open|High| Low|Close|Volume_BTC|Volume_Currency|Weighted_Price|          Date_Time|Year|Month|
+----+----+----+-----+----------+---------------+--------------+-------------------+----+-----+
|4.39|4.39|4.39| 4.39|0.45558087|   2.0000000193|          4.39|2011-12-31 07:52:00|2011|   12|
| NaN| NaN| NaN|  NaN|       NaN|            NaN|           NaN|2011-12-31 07:53:00|2011|   12|
| NaN| NaN| NaN|  NaN|       NaN|            NaN|           NaN|2011-12-31 07:54:00|2011|   12|
| NaN| NaN| NaN|  NaN|       NaN|            NaN|           NaN|2011-12-31 07:55:00|2011|   12|
| NaN| NaN| NaN|  NaN|       NaN|            NaN|           NaN|2011-12-31 07:56:00|2011|   12|
+----+----+----+-----+----------+---------------+--------------+-------------------+----+-----+
only showing top 5 rows



#  C. Using Parquet

<p align="justify">
<font size="3">
In order to gain in performance, it is a good idea to use a NoSQL structure, here Parquet, that will allow to partition the Spark Dataframe and to store it in multiple Parquet files. 
</font>
</p>

<p align="justify">
<font size="3">
Parquet is a columnar storage format available to any project in the Hadoop ecosystem, regardless of the choice of data processing framework, data model or programming language.
</font>
</p>

## C.1 Saving data in Parquet
    
Here, partition the file according to:
    
 <ul>
    <li> the year </li>
    <li> the month of the year </li>
</ul>
</font>
</p>



In [11]:
df_clean.write.partitionBy(["Year", "Month"]).parquet("BTC/", mode='overwrite')
print("writing to Parquet done")

writing to Parquet done


##  C.2 Check the folder Structure

 
<p align="justify">
<font size="3">
Look at the folder structure that has been created for the storage of the file. You see how the partitioning stategy of Parquet and the data distribution of Spark can be used, explicitely or implicitely, to improve performance.

While you navigate, remember that in the data access:
    
 <ul>
     <li> the navigation is done using Parquet </li>
     <li> the leafs contain the encoded Parquet files </li>
</ul>
</font>
</p>




In [12]:
#BTC
#        ├── Year=2011
#        │   ├── ...
#        │   │
#        │   ├── month=12
#        ├── Year=2012
#        │   ├── month=1
#        │   ├── ...
#        │   │
#       ...

This folder structure corresponds to a physical and logical data partitioning.

# D. Pandas

 
<p align="justify">
<font size="3">
This data organization opens the opportunity to read data also using Pandas and not using Parquet. Write the command that uses Pandas read the data for the year 2011.
    
</font>
</p>

In [13]:
df = pd.read_parquet("BTC/Year=2011")
df.head()

#######################
# Check the expected output:
#Open	High	Low	Close	Volume_BTC	Volume_Currency	Weighted_Price	Date_Time	Month
#0	4.39	4.39	4.39	4.39	0.455581	2.0	4.39	2011-12-31 07:52:00	12
#1	NaN	NaN	NaN	NaN	NaN	NaN	NaN	2011-12-31 07:53:00	12
#2	NaN	NaN	NaN	NaN	NaN	NaN	NaN	2011-12-31 07:54:00	12
#3	NaN	NaN	NaN	NaN	NaN	NaN	NaN	2011-12-31 07:55:00	12
#...

,Open,High,Low,Close,Volume_BTC,Volume_Currency,Weighted_Price,Date_Time,Month
0,4.39,4.39,4.39,4.39,0.455581,2.0,4.39,2011-12-31 07:52:00,12
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011-12-31 07:53:00,12
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011-12-31 07:54:00,12
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011-12-31 07:55:00,12
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011-12-31 07:56:00,12


###  D.1 Read Parquet file
    
<p align="justify">
<font size="3">
Here you can see now the the Spark DataFrame is created from Parquet data.
</font>
</p>

In [14]:
df_parquet = spark.read.parquet("BTC/")
print("reading done")

reading done


## <strong>Exercise 6</strong>. Verify number of columns and count the number of rows (2 points)
    
<p align="justify">
Maybe you have not noticed that the volume of the data we are treating is not as small as it seems. Count the number of rows in the dataframe.
<font size="3">
</font>
</p>

In [15]:
count = df_parquet.count()
print(count)

#######################
# Expected output:
# 4857377

4857377


In [16]:
len(df_parquet.columns)

10

So, the DataFrame has 4,857,377 rows and 10 columns. We can also check its schema.

In [17]:
df_parquet.printSchema()

root
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume_BTC: double (nullable = true)
 |-- Volume_Currency: double (nullable = true)
 |-- Weighted_Price: double (nullable = true)
 |-- Date_Time: timestamp (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)



# E. Statistics

<p align="justify">
<font size="3">
We want to calculate descriptive statistics on the bitcoin by month/year.

The computed statistics will be stored in a DataFrame having this schema
<ul>
     <li>   Mean_Vol  : double </li>
     <li>   Std_Vol   : double </li>
     <li>   Min_Vol   : double </li>
     <li>   Max_vol   : double </li>
     <li>   Year      : int </li>
     <li>   Month     : int </li>
  
</ul>

In this exercise you will develop two different methodologies to compute the statistics:
<ul>
    <li>   using the <code>applyInPandas()</code> Pyspark function and the Pandas functions </li>
     <li>  only using the Pyspark functionnalities </li>
</ul>
The statistics computed should be stored in a Pandas DataFrame with both approaches.
</font>
</p>

## E.1. Spark applyInPandas
<p align="justify">
<font size="3">
The solution with <code>applyInPandas</code>
</font>
</p>

In [18]:
def compute_stats(key,df):
    res = df["Volume_BTC"].describe()
    res_dict = {}
    for index, value in res.items():
        if index == "mean":
            res_dict["Mean_Vol"] = value
        elif index == "std":
            res_dict["Std_Vol"] = value
        elif index == "min":
            res_dict["Min_Vol"] = value
        elif index == "max":
            res_dict["Max_Vol"] = value

    final =  pd.DataFrame([res_dict])
    final["Year"]  = key[0]
    final["Month"] = key[1]
    return final

### <strong>Exercise 7</strong>. The two parameters of the Python function (2 points)

Explain in detail how the <code>applyInPandas(func, schema)</code> works and describe what the $key$ and the $df$ will contain in our example.

According to the documentation, each group of a grouped Spark DataFrame is passed as a pandas.DataFrame to a user-defined function <code>func</code>. The returned pandas.DataFrame objects (corresponding to each group) are combined into a single DataFrame, and the result is returned. It should be noted that the function requires a full shuffle; thus, all the data of a group will be loaded into memory.

The <code>schema</code> will determine the return type of the <code>func</code>. Said otherwise, <code>schema</code> shows the data type of the columns of the returned Spark DataFrame.

If a user-defined function takes two arguments, the grouping key(s) will be passed as the first argument, and the data will be passed as the second argument. The grouping key(s) will be passed as a tuple of numpy data types, and the data will be passed in as a pandas.DataFrame. In our example, the <code>key</code> is a tuple of (year, month) and the <code>df</code> is a pandas.DataFrame for each year-month group.

### <strong>Exercise 8</strong>. The two parameters in action (1 point)
<p align="justify">
<font size="3">
Compute the statistics using then the <code>applyInPandas</code> and the provided functions. 

</font>
</p>

In [19]:
%%time
schema = "Mean_Vol double, Std_Vol double, Min_Vol double, Max_Vol double, Year int, Month int"
stats_df = df_parquet.groupby("Year", "Month")\
                     .applyInPandas(compute_stats, schema)
stats_df.show(5)

####### EXPECTED OUTPUT
#+------------------+------------------+----------+------------+----+-----+
#|          Mean_Vol|           Std_Vol|   Min_Vol|     Max_Vol|Year|Month|
#+------------------+------------------+----------+------------+----+-----+
#| 20.39613620802532| 54.24699556644988|    9.4E-5|2258.8231405|2012|   10|
#|12.095179597807542|44.149334198665166| 2.0452E-4|2037.2239038|2015|    2|
#| 6.147061206279663|17.745599117954125|0.00127783|564.21436237|2019|   10|
#| 8.468866447160776|  28.9837002907642|    1.0E-8|1616.0600006|2017|    3|
#| 8.684880075589284| 17.69646210434965|       0.0|533.10078293|2017|    8|
#+------------------+------------------+----------+------------+----+-----+

+------------------+------------------+----------+------------+----+-----+
|          Mean_Vol|           Std_Vol|   Min_Vol|     Max_Vol|Year|Month|
+------------------+------------------+----------+------------+----+-----+
| 20.39613620802532| 54.24699556644988|    9.4E-5|2258.8231405|2012|   10|
|12.095179597807542|44.149334198665166| 2.0452E-4|2037.2239038|2015|    2|
| 6.147061206279663|17.745599117954125|0.00127783|564.21436237|2019|   10|
| 8.468866447160776|  28.9837002907642|    1.0E-8|1616.0600006|2017|    3|
| 8.684880075589284| 17.69646210434965|       0.0|533.10078293|2017|    8|
+------------------+------------------+----------+------------+----+-----+
only showing top 5 rows

CPU times: user 103 ms, sys: 20.9 ms, total: 123 ms
Wall time: 17 s


### <strong>Exercise 9</strong>. The statsdf DataFrame (1 point)
<p align="justify">
<font size="3">
Which kind of DataFrame is <code>stats_df</code>?
</font>
</p>

In [20]:
type(stats_df)

pyspark.sql.dataframe.DataFrame

As we can see, the given DataFrame is just a Spark DataFrame.

### <strong>Exercise 10</strong>. DataFrame in Pandas (2 points)

<p align="justify">
<font size="3">
Since we computed descriptive statistics by month/year, the results are small. (We have only one row for each month.) Therefore, we can get and handle all the results in memory in Pandas. Notice that Spark uses lazy evaluation; thus, we need the <code>toPandas</code> action to trigger the computation.    
</font>
</p>

In [21]:
stats_pandas = stats_df.toPandas()
stats_pandas.head(10)

#######################
# Expected output:
#	Mean_Vol	Std_Vol	Min_Vol	Max_Vol	Year	Month
#0	20.396136	54.246996	9.400000e-05	2258.823141	2012	10
#1	12.095180	44.149334	2.045200e-04	2037.223904	2015	2
#2	6.147061	17.745599	1.277830e-03	564.214362	2019	10
#3	8.468866	28.983700	1.000000e-08	1616.060001	2017	3
#4	8.684880	17.696462	0.000000e+00	533.100783	2017	8
#5	16.040933	57.641501	2.044000e-05	4111.876106	2014	4
#6	4.984386	18.903445	1.054000e-05	822.866974	2020	6
#7	8.331579	18.350084	5.758000e-04	806.636224	2019	5
#8	8.621910	18.820399	4.047000e-04	602.282607	2017	10
#9	3.106413	10.738051	3.300000e-06	582.564185	2018	10

,Mean_Vol,Std_Vol,Min_Vol,Max_Vol,Year,Month
0,20.396136,54.246996,9.400000e-05,2258.823141,2012,10
1,12.095180,44.149334,2.045200e-04,2037.223904,2015,2
2,6.147061,17.745599,1.277830e-03,564.214362,2019,10
3,8.468866,28.983700,1.000000e-08,1616.060001,2017,3
4,8.684880,17.696462,0.000000e+00,533.100783,2017,8
5,16.040933,57.641501,2.044000e-05,4111.876106,2014,4
6,4.984386,18.903445,1.054000e-05,822.866974,2020,6
7,8.331579,18.350084,5.758000e-04,806.636224,2019,5
8,8.621910,18.820399,4.047000e-04,602.282607,2017,10
9,3.106413,10.738051,3.300000e-06,582.564185,2018,10


###  <strong>Exercise 11</strong>. Show the stats of the stats (1 point)


<p align="justify">
<font size="3">
We want to calculate the statistics of the bitcoin by month for all the years.

The computed statistics will be stored in a DataFrame having this schema
<ul>
     <li>   the min of the set min values </li>
     <li>   the mean of the set of mean values </li>
     <li>   ... </li> 
</ul>


    
</font>
</p>


In [22]:
# stats_pandas.describe()\
#     .drop(labels=["25%", "50%", "75%"], axis="index")\
#     .rename(index={'std': 'stddev'})

stats_df.describe().show()

#######################
# Expected output:
#+-------+------------------+------------------+--------------------+------------------+------------------+------------------+
#|summary|          Mean_Vol|           Std_Vol|             Min_Vol|           Max_Vol|              Year|             Month|
#+-------+------------------+------------------+--------------------+------------------+------------------+------------------+
#|  count|               112|               112|                 112|               112|               112|               112|
#|   mean|10.782191354847754|28.871463944232485|0.004551177678571...|1067.2847720235718|2016.0892857142858| 6.428571428571429|
#| stddev| 6.488551661205522| 18.11344145463867|0.043048607639448476| 895.8083462469303|2.7164947320662614|3.5252353718985097|
#|    min| 2.929999689326444| 6.490701567379118|                 0.0|       43.31219578|              2011|                 1|
#|    max|31.504423573146152|106.97606692383131|          0.45558087|      5853.8521659|              2021|                12|
#+-------+------------------+------------------+--------------------+------------------+------------------+------------------+


+-------+------------------+------------------+--------------------+------------------+------------------+------------------+
|summary|          Mean_Vol|           Std_Vol|             Min_Vol|           Max_Vol|              Year|             Month|
+-------+------------------+------------------+--------------------+------------------+------------------+------------------+
|  count|               112|               112|                 112|               112|               112|               112|
|   mean|10.782191354847754|28.871463944232485|0.004551177678571...|1067.2847720235718|2016.0892857142858| 6.428571428571429|
| stddev| 6.488551661205522| 18.11344145463867|0.043048607639448476| 895.8083462469303|2.7164947320662614|3.5252353718985097|
|    min| 2.929999689326444| 6.490701567379118|                 0.0|       43.31219578|              2011|                 1|
|    max|31.504423573146152|106.97606692383131|          0.45558087|      5853.8521659|              2021|            

# F. Plotting and equivalence 
<p align="justify">
<font size="3">
We want to plot the results of the statistics by year and month using <code>Plotly</code>.
</font>
</p>



In [23]:
init_notebook_mode(connected=True)

In [24]:
def get_date_from_year_month(df):
    df["Date"] = pd.to_datetime(
        df['Year'].astype(str) + '-' + df['Month'].astype(str), format='%Y-%m')     

In [25]:
print(stats_pandas.columns)
get_date_from_year_month(stats_pandas)
print(stats_pandas.columns)

Index(['Mean_Vol', 'Std_Vol', 'Min_Vol', 'Max_Vol', 'Year', 'Month'], dtype='object')
Index(['Mean_Vol', 'Std_Vol', 'Min_Vol', 'Max_Vol', 'Year', 'Month', 'Date'], dtype='object')


In [26]:
stats_pandas.sort_values(by="Date", inplace=True)
stats_pandas.reset_index(inplace=True, drop=True)
stats_pandas

,Mean_Vol,Std_Vol,Min_Vol,Max_Vol,Year,Month,Date
0,23.829470,22.711133,0.455581,48.000000,2011,12,2011-12-01
1,4.031777,6.740555,0.020000,43.312196,2012,1,2012-01-01
2,8.313993,11.924511,0.003138,92.654874,2012,2,2012-02-01
3,15.197791,26.505990,0.002096,247.560124,2012,3,2012-03-01
4,21.683913,36.393538,0.001276,300.516098,2012,4,2012-04-01
...,...,...,...,...,...,...,...
107,6.695166,12.974320,0.001542,360.652808,2020,11,2020-11-01
108,5.742400,10.432593,0.001030,358.619032,2020,12,2020-12-01
109,10.253061,16.947097,0.000002,398.565948,2021,1,2021-01-01
110,5.965070,10.971809,0.000443,248.504517,2021,2,2021-02-01


In [27]:
mean_vol_trac = {
    "x": stats_pandas.Date,
    "y": stats_pandas["Mean_Vol"],
}

layout = {
  "height":1000,
  "showlegend": True, 
  "title": "Average Volume by Month of BTC",
}

fig = go.Figure(data=[mean_vol_trac], layout=layout)
fig.show(renderer="colab")

###  <strong> Exercise 12 </strong> - Compute the statistics using Pyspark (1 point)


<p align="justify">
<font size="3">
We want to calculate the statistics of the bitcoin as we did before. However, use PySpark methods this time.

To this end:
<ul>
     <li>   import the data from Parquet into a Spark DataFrame </li>
     <li>   remove null values </li>
     <li>   perform the aggregation of the results </li> 
     <li>   convert the results to Pandas </li> 
 
</ul> 
</font>
</p>

In [28]:
df_spark = spark.read.parquet("BTC/")

In [29]:
%%time
group_ym = df_spark.na.drop()\
    .select(["Volume_BTC", "Year", "Month"]).groupBy(["Year", "Month"])
res_df = group_ym.agg(
    mean("Volume_BTC").alias("Mean_Vol"),
    stddev("Volume_BTC").alias("Std_Vol"),
    spark_min("Volume_BTC").alias("Min_Vol"),
    spark_max("Volume_BTC").alias("Max_Vol"))
res_df.show(5)

+----+-----+------------------+------------------+----------+------------+
|Year|Month|          Mean_Vol|           Std_Vol|   Min_Vol|     Max_Vol|
+----+-----+------------------+------------------+----------+------------+
|2012|   10| 20.39613620802532| 54.24699556644981|    9.4E-5|2258.8231405|
|2015|    2|12.095179597807542| 44.14933419866533| 2.0452E-4|2037.2239038|
|2019|   10| 6.147061206279663| 17.74559911795412|0.00127783|564.21436237|
|2017|    3| 8.468866447160776|28.983700290764208|    1.0E-8|1616.0600006|
|2017|    8| 8.684880075589284|17.696462104349653|       0.0|533.10078293|
+----+-----+------------------+------------------+----------+------------+
only showing top 5 rows

CPU times: user 36 ms, sys: 4.17 ms, total: 40.1 ms
Wall time: 3.42 s


In [30]:
stats_pandas2 = res_df.toPandas()
get_date_from_year_month(stats_pandas2)    
stats_pandas2.sort_values(by="Date", inplace=True)
stats_pandas2.reset_index(inplace=True, drop=True)
stats_pandas2

,Year,Month,Mean_Vol,Std_Vol,Min_Vol,Max_Vol,Date
0,2011,12,23.829470,22.711133,0.455581,48.000000,2011-12-01
1,2012,1,4.031777,6.740555,0.020000,43.312196,2012-01-01
2,2012,2,8.313993,11.924511,0.003138,92.654874,2012-02-01
3,2012,3,15.197791,26.505990,0.002096,247.560124,2012-03-01
4,2012,4,21.683913,36.393538,0.001276,300.516098,2012-04-01
...,...,...,...,...,...,...,...
107,2020,11,6.695166,12.974320,0.001542,360.652808,2020-11-01
108,2020,12,5.742400,10.432593,0.001030,358.619032,2020-12-01
109,2021,1,10.253061,16.947097,0.000002,398.565948,2021-01-01
110,2021,2,5.965070,10.971809,0.000443,248.504517,2021-02-01


### Extra. Equivalence of results

<p align="justify">
<font size="3">
Now that you have seen two procedures to get the results, you must compare the outputs:
<ul>
     <li>  verify that the pandas.DataFrames resulting from <code>applyInPandas</code> and PySpark functions are equivalent. </li> 
      <li> compare the processing time between <code>applyInPandas</code> and PySpark functions and comment them. </li> 
 
</ul>
</font>
</p>



In [35]:
cols = ["Mean_Vol", "Std_Vol", "Min_Vol", "Max_Vol", "Year", "Month", "Date"]
try:
    assert_frame_equal(stats_pandas, stats_pandas2[cols])
    print("The two DataFrames are equal.")
except:  # if DataFrames are not equal
    print("The two DataFrames are NOT equal.")

The two DataFrames are equal.


### Comparison

1. <code>applyInPandas</code> method: CPU times: user 103 ms, sys: 20.9 ms, total: 123 ms, Wall time: 17 s.
2.   PySpark methods: CPU times: user 36 ms, sys: 4.17 ms, total: 40.1 ms, Wall time: 3.42 s.

It is obvious that using PySpark methods is 3-4 times faster than using <code>applyInPandas</code>. However, we should also note that <code>applyInPandas</code> is actually doing more work than the PySpark routine. In addition to <code>Mean_Vol</code>, <code>Std_Vol</code>, <code>Min_Vol</code>, and <code>Max_Vol</code>, <code>pd.DataFrame.describe()</code> method also computes 25th, 50th, and 75th percentiles of the <code>Volume_BTC</code>.

However, even if they did a similar amount of computation, I believe PySpark will still be faster (but not 3-4 times faster).


### Extra - Plotting the financial data (1 point)



<p align="justify">
<font size="3">
Now that you have seen some examples, you can draw your graphs:
<ul>
     <li>   filter the global DataFrame from Parquet and take only the first day of the year 2021 </li> 
         <li> convert it to a pandas.DataFrame </li> 
         <li> display the data using the <code>plot_candlestick</code> routine </li> 
 
</ul>


    
</font>
</p>



In [32]:
def plot_candlestick(df):
    trace = {
      "x": df.Date_Time,
      "close": dfp["Open"],
      "decreasing": {"line": {"color": "#008000"}}, 
      "high":df["High"] ,
      "increasing": {"line": {"color": "#db4052"}}, 
      "low": df["Low"],
      "name": "BTC", 
      "open": df["Close"],
      "type": "candlestick"
    }

    layout = {
      "height":1000,
      "showlegend": True, 
      "title": "Technical Analysis",
    }
    
    fig = go.Figure(data=[trace], layout=layout)
    fig.show(renderer="colab")

In [33]:
df_spark_filtered = df_spark.filter(df_spark.Date_Time == "2021-01-01")
dfp = df_spark_filtered.toPandas()

In [34]:
plot_candlestick(dfp)

### Conclusion 

<p align="justify">
<font size="3">
<code>applyInPandas</code>  can be very useful when you need to apply advanced Python code or Python libraries (e.g., scikit-learn). In other cases, however, you can just use PySpark routines that rely on powerful storage techniques (for example, Parquet).    
</font>
</p>